In [ ]:
# Cài đặt môi trường mỗi lẫn chạy với Colab
!sudo apt install tesseract-ocr
!pip install pytesseract vnstock
!sudo apt-get install tesseract-ocr-vie # Cài đặt gói ngôn ngữ tiếng Việt

# Một hình ảnh

In [ ]:
from vnstock.ocr import *
image_ocr(image_path=r'/content/chrome_runiB0dpB3.png', lang='vie', output_path=r'/content')

# File PDF

Demo này hoạt động tốt trên Google Colab và yêu cầu quá trình cài đặt ngắn gọn nhất. Nếu bạn muốn chạy trên môi trường máy tính Windows, hãy tham khảo hướng dẫn cài đặt chính thức từ Vnstock Docs [tại đây](https://docs.vnstock.site/integrate/pytesseract-ocr-chuyen-doi-tai-lieu-tai-chinh-scan-sang-van-ban/)

In [ ]:
!pip install PyPDF2 pdf2image
!sudo apt-get install poppler-utils

In [22]:
# @title Download report from Vietstock
from vnstock_data.vietstock import *

symbol = 'ACB'
report = financial_document_search(symbol=symbol, type=1).iloc[0]
url = report['Url']
name = report['Title']
r = requests.get(url)
with open(f'/content/{name}_{symbol}.pdf', 'wb') as f:
    f.write(r.content)

In [6]:
# @title PDF to Text - Pytesseract
import PyPDF2
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import os
from tqdm import tqdm

def extract_text_from_pdf(pdf_path, output_folder, lang='eng'):
    """Extracts text from each page of a PDF, saves it to separate files,
       and merges it into a single text file.

    Args:
        pdf_path (str): Path to the PDF file.
        output_folder (str): Path to the output folder for text files.
        lang (str, optional): Language code for OCR. Defaults to 'eng'.
    """

    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        os.makedirs(output_folder, exist_ok=True)  # Create output folder if needed

        for page_num in tqdm(range(num_pages), desc="Processing PDF pages"):
            # Extract images for each page
            images = convert_from_path(pdf_file.name, dpi=200, first_page=page_num + 1, last_page=page_num + 1)
            image = images[0]

            # Save image to file
            image_file = os.path.join(output_folder, f'page_{page_num}.png')
            image.save(image_file, 'PNG')

            # Perform OCR on the image
            text = pytesseract.image_to_string(image, lang=lang)

            # Save extracted text to a separate file
            text_file = os.path.join(output_folder, f'page_{page_num}.txt')
            with open(text_file, 'w') as f:
                f.write(text)

        # Merge extracted text into a single file
        with open(os.path.join(output_folder, 'extracted_text.txt'), 'w') as f:
            for text_file in os.listdir(output_folder):
                if text_file.endswith('.txt'):
                    with open(os.path.join(output_folder, text_file), 'r') as tf:
                        f.write(tf.read())

# Example usage:
pdf_path = '/content/ACB_Baocaotaichinh_Q3_2023_Congtyme.pdf'
output_folder = 'extract'
language = 'vie'  # Set the desired language for OCR

extract_text_from_pdf(pdf_path, output_folder, lang=language)


Processing PDF pages: 100%|██████████| 30/30 [03:39<00:00,  7.31s/it]
